In [2]:
from selenium import webdriver
import base64, pickle, time
from IPython.display import Image
from IPython.display import display
from PIL import Image as pil

### get youtube

In [110]:
driver =  webdriver.Chrome()
driver.get("https://www.youtube.com/watch?v=1oA_ap2UabU")

In [114]:
def saveScreen(filename):
    element = driver.find_element_by_css_selector('#movie_player')
    location = element.location
    size = element.size
    driver.save_screenshot('screenshot.png')
    im = pil.open('screenshot.png')
    left = location['x']
    top = location['y'] * 2
    right = left + size['width'] * 2
    bottom = top + size['height'] * 2

    # left, top, right, bottom
    im = im.crop((left, top, right, bottom)) 
    im.save(filename) 

In [115]:
saveScreen('screenshot2.png')